In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import warnings

warnings.filterwarnings("ignore")

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width))

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_height, img_width))

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Add classification head
num_classes = len(train_dataset.class_names)

model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])



# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Callbacks for early stopping and model checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 10
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=epochs,
                    callbacks=[early_stopping, reduce_lr])

# Load the best model
# model.load_weights('best_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

# Convert the model to TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the model
# with open('model_david.tflite', 'wb') as f:
#     f.write(tflite_model)

# print("Model converted to TFLite and saved as 'model.tflite'")

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 275ms/step - accuracy: 0.1072 - loss: 3.0029 - val_accuracy: 0.1608 - val_loss: 3.0196 - learning_rate: 1.0000e-04
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - accuracy: 0.6118 - loss: 1.5884 - val_accuracy: 0.2915 - val_loss: 2.9149 - learning_rate: 1.0000e-04
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 244ms/step - accuracy: 0.7998 - loss: 0.9491 - val_accuracy: 0.3920 - val_loss: 2.8943 - learning_rate: 1.0000e-04
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 238ms/step - accuracy: 0.8614 - loss: 0.6997 - val_accuracy: 0.4523 - val_loss: 2.8497 - learning_rate: 1.0000e-04
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 241ms/step - accuracy: 0.9163 - loss: 0.4864 - val_accuracy: 0.4724 - val_loss: 2.8435 - learning_rate: 1.0000e-04
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 246ms/step - accuracy: 0.9308 - loss: 0.3553 - val_accuracy: 0.4623 - val_loss: 2.9176 - learnin

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Simplified data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Learning rate scheduler
# def scheduler(epoch, lr):
#     if epoch < 5:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train the model
epochs = 30  # Keeping a higher number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 406ms/step - accuracy: 0.2571 - loss: 2.5605 - val_accuracy: 0.4824 - val_loss: 2.6670 - learning_rate: 0.0010
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 320ms/step - accuracy: 0.8246 - loss: 0.6559 - val_accuracy: 0.5829 - val_loss: 3.7654 - learning_rate: 0.0010
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 311ms/step - accuracy: 0.8961 - loss: 0.2813 - val_accuracy: 0.5477 - val_loss: 4.5148 - learning_rate: 0.0010
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 331ms/step - accuracy: 0.9393 - loss: 0.2023 - val_accuracy: 0.6080 - val_loss: 4.7385 - learning_rate: 0.0010
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 326ms/step - accuracy: 0.9636 - loss: 0.1214 - val_accuracy: 0.6131 - val_loss: 4.6073 - learning_rate: 2.0000e-04
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 351ms/step - accuracy: 0.9800 - loss: 0.0736 - val_accuracy: 0.6181 - val_loss: 4.5141 - learning_rate: 2.0000e-

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = tf.keras.applications.Xception(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
preprocess_input = tf.keras.applications.xception.preprocess_input

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Increased number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 53s 1us/step
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.0908 - loss: 2.9339 - val_accuracy: 0.2111 - val_loss: 2.6871 - learning_rate: 1.0000e-04
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.3662 - loss: 2.4441 - val_accuracy: 0.3869 - val_loss: 2.2846 - learning_rate: 1.0000e-04
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.6278 - loss: 1.7962 - val_accuracy: 0.5276 - val_loss: 2.0170 - learning_rate: 1.0000e-04
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7674 - loss: 1.1543 - val_accuracy: 0.5477 - val_loss: 2.2330 - learning_rate: 1.0000e-04
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.8492 - loss: 0.7371 - val_accuracy: 0.5528 - val_loss: 2.7471 - learning_rate: 1.0000e-04
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.8996 - loss: 0.4881 - val_accura

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = tf.keras.applications.NASNetMobile(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
preprocess_input = tf.keras.applications.nasnet.preprocess_input

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Increased number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 13s 1us/step
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 399ms/step - accuracy: 0.0670 - loss: 3.1310 - val_accuracy: 0.2161 - val_loss: 2.7786 - learning_rate: 1.0000e-04
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 232ms/step - accuracy: 0.2063 - loss: 2.6747 - val_accuracy: 0.3467 - val_loss: 2.5954 - learning_rate: 1.0000e-04
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 230ms/step - accuracy: 0.3395 - loss: 2.4005 - val_accuracy: 0.3618 - val_loss: 2.4528 - learning_rate: 1.0000e-04
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 231ms/step - accuracy: 0.4362 - loss: 2.1143 - val_accuracy: 0.3869 - val_loss: 2.3408 - learning_rate: 1.0000e-04
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 235ms/step - accuracy: 0.4835 - loss: 1.9560 - val_accuracy: 0.4171 - val_loss: 2.2589 - learning_rate: 1.0000e-04
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 267ms/step - accuracy: 0.5264 - loss: 1.6863

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 50  # Increased number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 399ms/step - accuracy: 0.0614 - loss: 4.4166 - val_accuracy: 0.1357 - val_loss: 2.8538 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 329ms/step - accuracy: 0.0956 - loss: 3.8067 - val_accuracy: 0.2613 - val_loss: 2.7184 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 354ms/step - accuracy: 0.1627 - loss: 3.3403 - val_accuracy: 0.3417 - val_loss: 2.5819 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 414ms/step - accuracy: 0.2265 - loss: 2.8671 - val_accuracy: 0.3668 - val_loss: 2.4538 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 471ms/step - accuracy: 0.2850 - loss: 2.5530 - val_accuracy: 0.4221 - val_loss: 2.3343 - learning_rate: 1.0000e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 459ms/step - accuracy: 0.3972 - loss: 2.0856 - val_accuracy: 0.4322 - val_loss: 2.2426 - lear

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB4
base_model = tf.keras.applications.EfficientNetB4(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze more layers of the base model for fine-tuning
base_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 300
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping, learning rate reduction, and learning rate scheduling
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 50  # Increased number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/50

25/25 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.0516 - loss: 4.4376 - val_accuracy: 0.1156 - val_loss: 2.9500 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.1087 - loss: 3.7369 - val_accuracy: 0.2060 - val_loss: 2.8411 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.1858 - loss: 3.0817 - val_accuracy: 0.2563 - val_loss: 2.6983 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.2835 - loss: 2.6071 - val_accuracy: 0.3467 - val_loss: 2.5447 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4404 - loss: 2.0342 - val_accuracy: 0.4271 - val_loss: 2.3678 - learning_rate: 1.0000e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.5059 - loss: 1.7096 - val_accuracy: 0.4724 - val_loss: 2.2190 - learning_rate: 1.00

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB4
base_model = tf.keras.applications.EfficientNetV2M(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze more layers of the base model for fine-tuning
base_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 200
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping, learning rate reduction, and learning rate scheduling
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 50
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 204s 5s/step - accuracy: 0.0729 - loss: 5.0828 - val_accuracy: 0.1357 - val_loss: 3.6326 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 137s 5s/step - accuracy: 0.1528 - loss: 4.2388 - val_accuracy: 0.2010 - val_loss: 3.4719 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 133s 5s/step - accuracy: 0.2308 - loss: 3.5996 - val_accuracy: 0.3467 - val_loss: 3.2548 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 104s 4s/step - accuracy: 0.4087 - loss: 2.8289 - val_accuracy: 0.3920 - val_loss: 3.0287 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 104s 4s/step - accuracy: 0.5771 - loss: 2.2169 - val_accuracy: 0.4673 - val_loss: 2.8482 - learning_rate: 1.0000e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 118s 5s/step - accuracy: 0.6629 - loss: 1.8437 - val_accuracy: 0.4724 - val_loss: 2.7561 - learning_rate:

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB4
base_model = tf.keras.applications.EfficientNetB4(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze more layers of the base model for fine-tuning
base_model.trainable = True
fine_tune_at = 200
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 ** (epoch / 20))

# Train the model
epochs = 50
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr, lr_scheduler]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.0607 - loss: 5.3127 - val_accuracy: 0.1106 - val_loss: 3.7266 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.0991 - loss: 4.6848 - val_accuracy: 0.2060 - val_loss: 3.6143 - learning_rate: 1.1220e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.1710 - loss: 3.9027 - val_accuracy: 0.2613 - val_loss: 3.4739 - learning_rate: 1.2589e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2843 - loss: 3.4032 - val_accuracy: 0.3266 - val_loss: 3.2984 - learning_rate: 1.4125e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.4377 - loss: 2.7295 - val_accuracy: 0.3920 - val_loss: 3.0891 - learning_rate: 1.5849e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5036 - loss: 2.4389 - val_accuracy: 0.4422 - val_loss: 2.8982 - learning_rate: 1.778

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.3),
    layers.RandomBrightness(0.3),
    layers.RandomTranslation(0.2, 0.2),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB4
base_model = tf.keras.applications.EfficientNetB4(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze more layers of the base model for fine-tuning
base_model.trainable = True
fine_tune_at = 200
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 ** (epoch / 20))

# Train the model
epochs = 50
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr, lr_scheduler]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')


Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.0696 - loss: 12.7296 - val_accuracy: 0.0553 - val_loss: 10.9226 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.0813 - loss: 12.2936 - val_accuracy: 0.0603 - val_loss: 10.8242 - learning_rate: 1.1220e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.0723 - loss: 11.9696 - val_accuracy: 0.1106 - val_loss: 10.6929 - learning_rate: 1.2589e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.1465 - loss: 11.5832 - val_accuracy: 0.2060 - val_loss: 10.4909 - learning_rate: 1.4125e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2056 - loss: 10.9186 - val_accuracy: 0.2965 - val_loss: 10.2208 - learning_rate: 1.5849e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.2877 - loss: 10.4467 - val_accuracy: 0.3970 - val_loss: 9.8717 - learning_